## Wainishaji wa vyakula 2

Katika somo hili la pili la uainishaji, tutachunguza `njia zaidi` za kuainisha data ya kategoria. Pia tutajifunza kuhusu athari za kuchagua mainishaji mmoja badala ya mwingine.

### [**Jaribio la awali la somo**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Mahitaji ya awali**

Tunadhani kuwa umekamilisha masomo ya awali kwa kuwa tutatumia baadhi ya dhana tulizojifunza hapo kabla.

Kwa somo hili, tutahitaji vifurushi vifuatavyo:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) ni [mkusanyiko wa vifurushi vya R](https://www.tidyverse.org/packages) vilivyoundwa ili kufanya sayansi ya data kuwa ya haraka, rahisi, na ya kufurahisha!

-   `tidymodels`: Mfumo wa [tidymodels](https://www.tidymodels.org/) ni [mkusanyiko wa vifurushi](https://www.tidymodels.org/packages/) kwa ajili ya uundaji wa mifano na ujifunzaji wa mashine.

-   `themis`: [Kifurushi cha themis](https://themis.tidymodels.org/) kinatoa Hatua za Ziada za Mapishi kwa Kushughulikia Data Isiyosawazishwa.

Unaweza kuvifunga kwa kutumia:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Vinginevyo, script iliyo hapa chini hukagua kama una vifurushi vinavyohitajika kukamilisha moduli hii na kuvifunga kwako endapo havipo.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Ramani ya uainishaji**

Katika [somo letu la awali](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1), tulijaribu kujibu swali: tunachaguaje kati ya mifano mbalimbali? Kwa kiasi kikubwa, inategemea sifa za data na aina ya tatizo tunalotaka kutatua (kwa mfano, uainishaji au regression?)

Hapo awali, tulijifunza kuhusu chaguo mbalimbali unazoweza kutumia unapouainisha data kwa kutumia karatasi ya msaada ya Microsoft. Mfumo wa Kujifunza kwa Mashine wa Python, Scikit-learn, unatoa karatasi ya msaada inayofanana lakini ya kina zaidi ambayo inaweza kusaidia zaidi kupunguza chaguo zako za estimators (neno lingine kwa classifiers):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Kidokezo: [tembelea ramani hii mtandaoni](https://scikit-learn.org/stable/tutorial/machine_learning_map/) na bonyeza kwenye njia ili kusoma nyaraka.
>
> Tovuti ya [Tidymodels reference](https://www.tidymodels.org/find/parsnip/#models) pia inatoa nyaraka bora kuhusu aina tofauti za modeli.

### **Mpango** 🗺️

Ramani hii ni muhimu sana mara tu unapokuwa na uelewa mzuri wa data yako, kwani unaweza 'kutembea' kwenye njia zake kuelekea uamuzi:

-   Tuna sampuli \>50

-   Tunataka kutabiri kategoria

-   Tuna data yenye lebo

-   Tuna sampuli chini ya 100K

-   ✨ Tunaweza kuchagua Linear SVC

-   Ikiwa hiyo haifanyi kazi, kwa kuwa tuna data ya nambari

    -   Tunaweza kujaribu ✨ KNeighbors Classifier

        -   Ikiwa hiyo haifanyi kazi, jaribu ✨ SVC na ✨ Ensemble Classifiers

Hii ni njia muhimu sana ya kufuata. Sasa, hebu tuanze moja kwa moja kwa kutumia mfumo wa modeli wa [tidymodels](https://www.tidymodels.org/): mkusanyiko thabiti na rahisi wa pakiti za R zilizotengenezwa ili kuhimiza mazoea mazuri ya takwimu 😊.

## 2. Gawanya data na kushughulikia seti ya data isiyo na uwiano.

Kutoka kwenye masomo yetu ya awali, tulijifunza kuwa kulikuwa na seti ya viungo vya kawaida katika vyakula vyetu. Pia, kulikuwa na usambazaji usio sawa katika idadi ya vyakula.

Tutashughulikia haya kwa:

-   Kuondoa viungo vya kawaida zaidi vinavyosababisha mkanganyiko kati ya vyakula tofauti, kwa kutumia `dplyr::select()`.

-   Kutumia `recipe` inayosindika data ili kuifanya iwe tayari kwa modeli kwa kutumia algoriti ya `over-sampling`.

Tayari tulitazama haya katika somo la awali kwa hivyo hili linapaswa kuwa rahisi 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Kushughulikia Data Isiyosawazishwa

Data isiyosawazishwa mara nyingi ina athari mbaya kwenye utendaji wa modeli. Modeli nyingi hufanya kazi vizuri zaidi pale idadi ya uchunguzi ni sawa, na kwa hivyo huwa zinapata changamoto na data isiyosawazishwa.

Kuna njia kuu mbili za kushughulikia seti za data isiyosawazishwa:

-   kuongeza uchunguzi kwenye darasa lenye idadi ndogo: `Over-sampling` kwa mfano kutumia algoriti ya SMOTE ambayo huzalisha mifano mipya ya darasa lenye idadi ndogo kwa kutumia majirani wa karibu wa kesi hizo.

-   kuondoa uchunguzi kutoka darasa lenye idadi kubwa: `Under-sampling`

Katika somo letu la awali, tulionyesha jinsi ya kushughulikia seti za data isiyosawazishwa kwa kutumia `recipe`. Recipe inaweza kufikiriwa kama mpango unaoelezea hatua gani zinapaswa kutumika kwenye seti ya data ili kuifanya iwe tayari kwa uchambuzi wa data. Katika hali yetu, tunataka kuwa na usambazaji sawa wa idadi ya vyakula vyetu kwa `training set` yetu. Hebu tuingie moja kwa moja.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Sasa tuko tayari kufundisha mifano 👩‍💻👨‍💻!

## 3. Zaidi ya mifano ya regression ya multinomial

Katika somo letu la awali, tulichunguza mifano ya regression ya multinomial. Hebu tuangalie mifano mingine yenye kubadilika zaidi kwa ajili ya uainishaji.

### Support Vector Machines

Katika muktadha wa uainishaji, `Support Vector Machines` ni mbinu ya kujifunza kwa mashine inayojaribu kutafuta *hyperplane* inayotenganisha darasa kwa "ubora" zaidi. Hebu tuangalie mfano rahisi:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ haigawanyi madarasa. H2~ inagawanya, lakini kwa pengo dogo tu. H3~ inagawanya kwa pengo kubwa zaidi.

#### Klasifaya ya Msaada wa Vector ya Mstari

Kuweka vikundi kwa kutumia Support-Vector (SVC) ni sehemu ya familia ya mbinu za ML za Support-Vector machines. Katika SVC, hyperplane huchaguliwa ili kutenganisha kwa usahihi `sehemu kubwa` ya uchunguzi wa mafunzo, lakini `inaweza kukosea` uchunguzi kadhaa. Kwa kuruhusu baadhi ya alama kuwa upande usio sahihi, SVM inakuwa thabiti zaidi kwa data isiyo ya kawaida na hivyo kuboresha uwezo wa kujumlisha data mpya. Kigezo kinachosimamia ukiukaji huu kinaitwa `gharama` ambayo ina thamani ya msingi ya 1 (tazama `help("svm_poly")`).

Hebu tuunde SVC ya mstari kwa kuweka `degree = 1` katika mfano wa polynomial SVM.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Sasa kwa kuwa tumeshakamata hatua za awali za uchakataji na maelezo ya modeli ndani ya *workflow*, tunaweza kuendelea na kufundisha SVC ya mstari na kutathmini matokeo wakati huo huo. Kwa vipimo vya utendaji, hebu tuunde seti ya vipimo ambayo itatathmini: `accuracy`, `sensitivity`, `Positive Predicted Value` na `F Measure`.

> `augment()` itaongeza safu(safu) za utabiri kwenye data iliyotolewa.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Mashine ya Msaada wa Vector

Mashine ya msaada wa vector (SVM) ni upanuzi wa mclasifia wa msaada wa vector ili kuweza kushughulikia mpaka usio wa mstari kati ya madarasa. Kimsingi, SVM hutumia *mbinu ya kernel* kupanua nafasi ya sifa ili kuendana na uhusiano usio wa mstari kati ya madarasa. Mojawapo ya kazi maarufu na yenye kubadilika sana ya kernel inayotumiwa na SVM ni *Radial basis function.* Hebu tuone jinsi itakavyofanya kazi kwenye data yetu.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Bora zaidi 🤩!

> ✅ Tafadhali angalia:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Hands-on Machine Learning with R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), An Introduction to Statistical Learning with Applications in R
>
> kwa kusoma zaidi.

### Vainishi vya Jirani wa Karibu

*K*-jirani wa karibu (KNN) ni algorithimu ambapo kila uchunguzi unatabiriwa kulingana na *ufanano* wake na uchunguzi mwingine.

Hebu tuifanyie data yetu.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Inaonekana kwamba modeli hii haifanyi kazi vizuri sana. Huenda kubadilisha vigezo vya modeli (tazama `help("nearest_neighbor")`) kutaboresha utendaji wa modeli. Hakikisha kujaribu.

> ✅ Tafadhali angalia:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> ili kujifunza zaidi kuhusu *K*-Nearest Neighbors classifiers.

### Wahesabuji wa Ensemble

Algoriti za ensemble hufanya kazi kwa kuunganisha makadirio kadhaa ya msingi ili kuzalisha modeli bora kwa kutumia:

`bagging`: kutumia *kazi ya wastani* kwa mkusanyiko wa modeli za msingi

`boosting`: kujenga mfululizo wa modeli zinazojenga juu ya kila moja ili kuboresha utendaji wa utabiri.

Hebu tuanze kwa kujaribu modeli ya Random Forest, ambayo hujenga mkusanyiko mkubwa wa miti ya maamuzi kisha hutumia kazi ya wastani ili kupata modeli bora zaidi kwa ujumla.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Kazi nzuri 👏!

Hebu pia tujaribu na mfano wa Boosted Tree.

Boosted Tree hufafanua mbinu ya ensemble inayounda mfululizo wa miti ya maamuzi ya mfululizo ambapo kila mti unategemea matokeo ya miti ya awali kwa lengo la kupunguza makosa hatua kwa hatua. Inalenga uzito wa vitu vilivyokosewa kuainishwa na kurekebisha mwelekeo wa classifier inayofuata ili kusahihisha.

Kuna njia tofauti za kufanikisha mfano huu (tazama `help("boost_tree")`). Katika mfano huu, tutafanikisha Boosted trees kupitia injini ya `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

✅ Tafadhali angalia:

-   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)

-   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)

-   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)

-   <https://algotech.netlify.app/blog/xgboost/> - Inachunguza mfano wa AdaBoost ambao ni mbadala mzuri kwa xgboost.

kujifunza zaidi kuhusu waainishaji wa Ensemble.

## 4. Ziada - kulinganisha mifano mingi

Tumetengeneza idadi kubwa ya mifano katika maabara hii 🙌. Inaweza kuwa kazi ngumu au ya kuchosha kuunda mtiririko wa kazi nyingi kutoka kwa seti tofauti za preprocessors na/au maelezo ya mifano kisha kuhesabu vipimo vya utendaji moja baada ya nyingine.

Hebu tuone kama tunaweza kushughulikia hili kwa kuunda kazi ambayo inafaa orodha ya mtiririko wa kazi kwenye seti ya mafunzo kisha inarudisha vipimo vya utendaji kulingana na seti ya majaribio. Tutatumia `map()` na `map_dfr()` kutoka kwenye kifurushi cha [purrr](https://purrr.tidyverse.org/) ili kutumia kazi kwa kila kipengele katika orodha.

> [`map()`](https://purrr.tidyverse.org/reference/map.html) kazi zinakuruhusu kubadilisha mikondo mingi ya for na msimbo ambao ni mfupi zaidi na rahisi kusoma. Sehemu bora ya kujifunza kuhusu [`map()`](https://purrr.tidyverse.org/reference/map.html) kazi ni sura ya [iteration](http://r4ds.had.co.nz/iteration.html) katika R kwa data science.


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


Kifurushi cha [**workflowset**](https://workflowsets.tidymodels.org/) kinawawezesha watumiaji kuunda na kufanikisha urahisi idadi kubwa ya mifano ya modeli, lakini kimeundwa hasa kufanya kazi na mbinu za sampuli kama `cross-validation`, mbinu ambayo bado hatujafikia.

## **🚀Changamoto**

Kila moja ya mbinu hizi ina idadi kubwa ya vigezo ambavyo unaweza kurekebisha, kwa mfano `cost` katika SVMs, `neighbors` katika KNN, `mtry` (Vitabiri Vilivyochaguliwa kwa Nasibu) katika Random Forest.

Fanya utafiti kuhusu vigezo vya msingi vya kila moja na fikiria maana ya kurekebisha vigezo hivi kwa ubora wa modeli.

Ili kupata maelezo zaidi kuhusu modeli fulani na vigezo vyake, tumia: `help("model")` mfano `help("rand_forest")`

> Kwa vitendo, mara nyingi tunafanya *makadirio* ya *thamani bora* kwa kufundisha modeli nyingi kwenye `seti ya data iliyosimuliwa` na kupima jinsi modeli hizi zinavyofanya kazi. Mchakato huu unaitwa **tuning**.

### [**Jaribio la baada ya somo**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Mapitio na Kujifunza Binafsi**

Kuna maneno mengi ya kitaalamu katika masomo haya, kwa hivyo chukua muda kupitia [orodha hii](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) ya istilahi muhimu!

#### SHUKRANI KWA:

[`Allison Horst`](https://twitter.com/allison_horst/) kwa kuunda michoro ya kuvutia inayofanya R kuwa ya kupendeza na ya kuvutia zaidi. Pata michoro zaidi kwenye [galeria yake](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Cassie Breviu](https://www.twitter.com/cassieview) na [Jen Looper](https://www.twitter.com/jenlooper) kwa kuunda toleo la awali la moduli hii kwa Python ♥️

Jifunze kwa furaha,

[Eric](https://twitter.com/ericntay), Balozi wa Wanafunzi wa Microsoft Learn wa Dhahabu.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Michoro na @allison_horst</figcaption>



---

**Kanusho**:  
Hati hii imetafsiriwa kwa kutumia huduma ya tafsiri ya AI [Co-op Translator](https://github.com/Azure/co-op-translator). Ingawa tunajitahidi kwa usahihi, tafadhali fahamu kuwa tafsiri za kiotomatiki zinaweza kuwa na makosa au kutokuwa sahihi. Hati ya asili katika lugha yake ya awali inapaswa kuzingatiwa kama chanzo cha mamlaka. Kwa taarifa muhimu, inashauriwa kutumia tafsiri ya kitaalamu ya binadamu. Hatutawajibika kwa maelewano mabaya au tafsiri zisizo sahihi zinazotokana na matumizi ya tafsiri hii.
